In [ ]:
import pandas as pd
import py_stringmatching as sm
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
from sklearn.model_selection import train_test_split
import re
import operator
style.use('ggplot')

# read in csv as dataframe
c = pd.read_csv("../blocking/candidate_set.csv", encoding="ISO-8859-1", index_col='_id')

c.head()

In [ ]:
matches = 0
nonmatches = 0;

gold = None
DS = None

while matches < 50:
    label = train_test_split(c, train_size=0.1039, test_size=0.8961)
    gold = label['train']
    unlabeled = label['test']

    #Automatically label the training data
    labels = []
    matches = 0;
    nonmatches = 0;
    for index, row in gold.iterrows():
        if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
            labels.append(1)
            matches = matches + 1
        else:
            labels.append(0)
            nonmatches = nonmatches + 1

gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

In [ ]:
gold.to_csv('./gold.csv')
unlabeled.to_csv('./unlabeled.csv')

In [ ]:
# remove unnecessary features
# write out gold and unlabeled csvs with only selected features for matcher

cols_to_keep = ['country_count_sim', 
                'status_match', 
                'country_overlap', 
                'name_name_jac_dlm_dc0_dlm_dc0', 
                'name_name_jac_qgm_3_qgm_3']

gold_reduced = gold[cols_to_keep + ['labels']] 
unlabeled_reduced = unlabeled[cols_to_keep]

In [ ]:
gold_reduced.to_csv('./gold_reduced.csv')
unlabeled_reduced.to_csv('./unlabeled_reduced.csv')

In [ ]:
#using features for matcher
test_train = train_test_split(gold, train_size=0.7, test_size=0.3)
train = test_train['train']
test = test_train['test']
len(train)

train_reduced = train[cols_to_keep + ['labels']] 
test_reduced = test[cols_to_keep + ['labels']]

In [ ]:
train.to_csv('./train.csv')
test.to_csv('./test.csv')
train_reduced.to_csv('./train_reduced.csv')
test_reduced.to_csv('./test_reduced.csv')